In [80]:
import pandas as pd
import numpy as np  
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split
from collections import defaultdict

pd.set_option('display.float_format', lambda x: '%.0f' % x)

In [ ]:
data_lake_prd_314410_cz_canais = pd.read_csv('c:\\Users\\paulo\\Desktop\\hotels brazil\\data\\lookups\\data-lake-prd-314410.cz.canais.csv')
data_lake_prd_314410_cz_canais

In [ ]:
data_lake_prd_314410_cz_canais.groupby('Produto').size().reset_index(name='Counts').sort_values(by='Counts', ascending=False)

In [ ]:
data_lake_prd_314410_cz_canais.groupby('Tipo').size().reset_index(name='Counts').sort_values(by='Counts', ascending=False)

In [ ]:
data_lake_prd_314410_cz_canais.groupby('Segmento').size().reset_index(name='Counts').sort_values(by='Counts', ascending=False)

In [ ]:
data_lake_prd_314410_cz_canais.groupby('Status').size().reset_index(name='Counts').sort_values(by='Counts', ascending=False)

In [ ]:
data_lake_prd_314410_cz_canais.groupby('Modelo').size().reset_index(name='Counts').sort_values(by='Counts', ascending=False)

In [ ]:
data_lake_prd_314410_cz_canais.groupby('Categoria').size().reset_index(name='Counts').sort_values(by='Counts', ascending=False)

In [ ]:
data_lake_prd_314410_cz_canais.groupby('Categoria_ID').size().reset_index(name='Counts').sort_values(by='Counts', ascending=False)

In [ ]:
data_lake_prd_314410_cz_canais.groupby('Modelo_ID').size().reset_index(name='Counts').sort_values(by='Counts', ascending=False)

In [ ]:
hotel_city_chanel_combin_extract  = pd.read_csv('c:\\Users\\paulo\\Desktop\\hotels brazil\\data\\other\\hotel_city_chanel_combin_extract.csv')
hotel_city_chanel_combin_extract.dropna(inplace=True)
hotel_city_chanel_combin_extract.drop(columns=['Cidade_ID'], inplace=True)
hotel_city_chanel_combin_extract.drop_duplicates(inplace=True)
hotel_city_chanel_combin_extract

In [ ]:
# Pivot the table
pivot_table = hotel_city_chanel_combin_extract.pivot_table(index='Hotel_ID', columns='Canal_ID', aggfunc='size', fill_value=0)

# Convert the table to binary (1 where the combination existed, 0 otherwise)
pivot_table = pivot_table.map(lambda x: 1 if x > 0 else 0)

pivot_table

In [ ]:
# Count the number of 1s for each column
counts_per_channel = pivot_table.sum().sort_values(ascending=False)
counts_per_channel

In [ ]:
pivot_table.loc[0:10:,0:10]

In [ ]:
def prepare_data_test(df):
    ratings = []
    for hotel in df.index:
        for channel in df.columns:
            ratings.append((hotel, channel, df.loc[hotel, channel]))
                          
    return(ratings)

prepare_data_test(pivot_table.loc[0:10:,0:10])

In [44]:
# Step 1: Prepare the data matrix
def prepare_data(df):
    ratings = []
    for hotel in df.index:
        for channel in df.columns:
            ratings.append((hotel, channel, df.loc[hotel, channel]))

    ratings_df = pd.DataFrame(ratings, columns=['hotel', 'channel', 'rating'])
    reader = Reader(rating_scale=(0, 1))
    return Dataset.load_from_df(ratings_df, reader)


In [45]:
# Step 2: Train the model
def train_model(data):
    trainset, testset = train_test_split(data, test_size=0.25, random_state=42)
    model = SVD(n_factors=100, n_epochs=20, lr_all=0.005, reg_all=0.02)
    model.fit(trainset)
    return model


In [46]:
# Step 3: Generate recommendations
def get_top_n_recommendations(predictions, n=5):
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))
    
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
    
    return top_n

In [47]:
# Step 4: Recommend channels
def recommend_channels(hotel_id, model, data, n=5):
    iids = data.df['channel'].unique()
    testset = [(hotel_id, iid, 0) for iid in iids]
    predictions = model.test(testset)
    top_n = get_top_n_recommendations(predictions, n)
    return top_n[hotel_id]

In [52]:
# Main execution
df = pivot_table
data = prepare_data(df)
model = train_model(data)


In [ ]:

# Example usage
hotel_id = df.index[52]  # Choose a hotel

recommendations = recommend_channels(hotel_id, model, data, n=50)

print(f"Recommended channels for hotel {hotel_id:.0f}:")

for channel, score in recommendations:
    print(f"{channel}: {score:.4f}")

In [ ]:
1145
810
104
791
908
1195
1168
1234
440
833
608
903
124
1181
71
607
396
594
256
201

In [ ]:
hotel_city_chanel_combin_extract  = pd.read_csv('c:\\Users\\paulo\\Desktop\\hotels brazil\\data\\other\\hotel_city_chanel_combin_extract.csv')
hotel_city_chanel_combin_extract[hotel_city_chanel_combin_extract['Hotel_ID']==413]

In [ ]:
data_lake_prd_314410_cz_canais[data_lake_prd_314410_cz_canais['Canal_ID'] == 496 ]

In [78]:
# Define the vector of Canal_IDs
canal_ids = [496, 810, 1145]


In [ ]:
data_lake_prd_314410_cz_canais[data_lake_prd_314410_cz_canais['Canal_ID'].isin(canal_ids)]
